In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv",nrows=300000)


In [3]:
df.sample()

,sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,...,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
170314,train_170314,213.267827,223.953356,235.017349,247.654629,258.578194,265.37908,263.849613,254.892024,243.128197,...,-9.388880e-07,6.568061e-07,0.0,345.531056,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# df.iloc[:,0:557]
df["sample_id"].nunique()

300000

In [5]:
# df.iloc[:,557:]

In [6]:
# data=df[["state_t","state_t_1","state_t_2","state_t_3","state_u",'state_v', 'state_ps', 'pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX', 'pbuf_TAUX', 'pbuf_TAUY', 'pbuf_COSZRS', 'cam_in_ALDIF', 'cam_in_ALDIR', 'cam_in_ASDIF', 'cam_in_ASDIR', 'cam_in_LWUP', 'cam_in_ICEFRAC', 'cam_in_LANDFRAC', 'cam_in_OCNFRAC', 'cam_in_SNOWHLAND', 'pbuf_ozone', 'pbuf_CH4', 'pbuf_N2O']]
data=df.iloc[:,0:557].drop(columns="sample_id")
target=df.iloc[:,557:]

In [7]:
# target=df[["ptend_t","ptend_q0001","ptend_q0002","ptend_q0003","ptend_u","ptend_v","cam_out_NETSW","cam_out_FLWDS","cam_out_PRECSC","cam_out_PRECC","cam_out_SOLS","cam_out_SOLL","cam_out_SOLSD","cam_out_SOLLD"]]

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [9]:
# random_reg=RandomForestRegressor()
# multi_reg=MultiOutputRegressor(random_reg)
# multi_reg.fit(X_train,Y_train)
# multi_reg.score(X_test,Y_test)

# # Deep learning

In [10]:
from keras.layers import Input,Dense,Dropout,LSTM,SimpleRNN,GRU,BatchNormalization,TimeDistributed
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2,l1_l2
from keras.callbacks import EarlyStopping

In [11]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1)

In [12]:
X_train_reshaped=X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])
X_test_reshaped=X_test.values.reshape(X_test.shape[0],1,X_test.shape[1])

In [15]:
inputs=Input(shape=(1,556))
S=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
S=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)

B=BatchNormalization()(S)

L=LSTM(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.2)(B)

L=LSTM(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
L=LSTM(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(L)

B=BatchNormalization()(L)
D=Dropout(0.2)(B)

G=GRU(128,activation='relu',return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
G=GRU(128,activation='relu',return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(G)
G=GRU(128,activation='relu',return_sequences=False,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(G)
B=BatchNormalization()(G)

D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(1)(D)
model=Model(inputs,outputs)
model.compile(optimizer="adam",loss='mse',metrics=['mae'])

In [16]:
model.fit(X_train_reshaped,Y_train,epochs=100,batch_size=32,validation_data=(X_test_reshaped,Y_test),callbacks=[early_stopping])

Epoch 1/100


I0000 00:00:1725264012.711034     836 service.cc:145] XLA service 0x58b00cb4cd60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725264012.711076     836 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725264012.711081     836 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  28/7500 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 738.7938 - mae: 3.2806   

I0000 00:00:1725264024.897114     836 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_subtract_fusion_11', 44 bytes spill stores, 52 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 44 bytes spill stores, 44 bytes spill loads

I0000 00:00:1725264024.915605     836 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7500/7500 ━━━━━━━━━━━━━━━━━━━━ 52s 4ms/step - loss: 695.3908 - mae: 3.6985 - val_loss: 726.2751 - val_mae: 9.6085
Epoch 2/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 686.7015 - mae: 3.7086 - val_loss: 691.4810 - val_mae: 3.6979
Epoch 3/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 687.9424 - mae: 3.7159 - val_loss: 691.4683 - val_mae: 3.7837
Epoch 4/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - loss: 688.7626 - mae: 3.7229 - val_loss: 691.4658 - val_mae: 3.7166
Epoch 5/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - loss: 688.6536 - mae: 3.7223 - val_loss: 691.4684 - val_mae: 3.6790
Epoch 6/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 687.9507 - mae: 3.7162 - val_loss: 691.4705 - val_mae: 3.6642
Epoch 7/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - loss: 690.5865 - mae: 3.7253 - val_loss: 691.4653 - val_mae: 3.7125
Epoch 8/100
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - loss: 688.4254 - mae: 3.7204 - val_loss: 691.4659 - val_mae: 3.7579
Epoc

In [17]:
loss,mae=model.evaluate(X_test_reshaped,Y_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 687.9539 - mae: 3.6807


In [18]:
for layer in model.layers:
    layer.trainable=False
model_output=model.layers[-7].output

# # M2

In [ ]:
# inputs=Input(shape=(1,556))
S=SimpleRNN(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(model_output)
S=SimpleRNN(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=SimpleRNN(64,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)

B=BatchNormalization()(S)
D=Dropout(0.2)(B)
L=LSTM(128,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
L=LSTM(128,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(L)
L=LSTM(128,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(L)

B=BatchNormalization()(L)
D=Dropout(0.2)(B)

G=GRU(256,activation='relu',return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
G=GRU(256,activation='relu',return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(G)
G=GRU(256,activation='relu',return_sequences=False,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(G)
B=BatchNormalization()(G)

D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs2=Dense(1)(D)
model2=Model(inputs=model.input,outputs=outputs2)
model2.compile(optimizer="adam",loss='mse',metrics=['mae'])

In [ ]:
model2.fit(X_train_reshaped,Y_train,epochs=100,batch_size=32,validation_data=(X_test_reshaped,Y_test),callbacks=[early_stopping])